In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

### <font color=blue> CovidHub ensemble state predictions

In [2]:
#####################################################
## COVID HUB ENSEMBLE STATE PREDICTIONS ############
#####################################################

import glob
import os
from tqdm.notebook import tqdm 

my_dir = os.path.join(DATA_DIR,'raw','COVIDhub-ensemble')
files = glob.glob(f'{my_dir}/*.csv')

types = ['point']
targets = ['1 wk ahead inc death']

all_dfs = []
for f in tqdm(files,total=len(files)):
    df = pd.read_csv(f)
    ## Choose which types (only point for now)
    df = df[df.type=='point']
    ## Only 1 wk ahead inc
    df = df[df.target.isin(targets)]
    all_dfs.append(df)
df_all = pd.concat(all_dfs)
# df_all.sort_values(['Fips','Date'], inplace=True)
df_all.reset_index(drop=True,inplace=True)
# display(df_all)

## check for nans
df = drop_sers_with_nans(df_all, from_axis='cols')

## REFORMAT dataframe ...
## rename cols
df.rename({'location':'State_fips',
           'target_end_date':'Date',
           'value':'Proj_inc_deaths'}, axis=1, inplace=True)
## choose cols
df = df[['State_fips','Date','Proj_inc_deaths']]

## IMPORTANT: choose only state fips
df = df[df.State_fips!='US']

## convert all cols to numerical
df['State_fips'] = df.State_fips.astype(int)
df['Date'] = df.Date.apply(lambda x: date_str_to_int(x))

## reset index
df.reset_index(inplace=True,drop=True)

## SAVE CSV!
# fpath = os.path.join(DATA_DIR,'processed','scratch',f'formatted_COVIDhub-ensemble_{DT}.csv')
fpath = os.path.join(DATA_DIR,'processed',f'formatted_COVIDhub-ensemble_{DT}.csv')
df.to_csv(fpath,index=False)
print(DT)

## READ BACK AND CHECK
dff = pd.read_csv(fpath)
display(df, dff)
dff.dtypes

  0%|          | 0/210 [00:00<?, ?it/s]

Dropped cols with NaNs!
Num cols before: 7
Num cols after: 6
20241001-223952


,State_fips,Date,Proj_inc_deaths
0,66,20200613,0.084503
1,69,20200613,0.410538
2,72,20200613,4.539067
3,78,20200613,0.123634
4,11,20200613,22.548359
...,...,...,...
8054,72,20230311,19.000000
8055,60,20230311,0.000000
8056,66,20230311,0.000000
8057,69,20230311,0.000000


,State_fips,Date,Proj_inc_deaths
0,66,20200613,0.084503
1,69,20200613,0.410538
2,72,20200613,4.539067
3,78,20200613,0.123634
4,11,20200613,22.548359
...,...,...,...
8054,72,20230311,19.000000
8055,60,20230311,0.000000
8056,66,20230311,0.000000
8057,69,20230311,0.000000


State_fips           int64
Date                 int64
Proj_inc_deaths    float64
dtype: object

### <font color=blue> NYTimes true county deaths



In [6]:
#####################################################
## NYT TRUE COUNTY DEATHS ###########################
#####################################################

fpath = os.path.join(DATA_DIR,'raw','nytimes','us-counties.csv')
df = pd.read_csv(fpath)

## check for rows with nans. 
# display(df)
df = drop_sers_with_nans(df, from_axis='rows')
# df.reset_index(drop=True, inplace=True)
# display(df)

## REFORMAT dataframe ...
## capitalize cols.
df.columns = df.columns.str.capitalize()
## rename cols
df.rename({'Deaths':'True_cum_deaths'},axis=1,inplace=True)
## choose cols
df = df[['Fips','Date', 'True_cum_deaths']]

## convert all cols to numerical
df['Fips'] = df.Fips.astype(int)
df['Date'] = df.Date.apply(lambda x: date_str_to_int(x))

## IMPORTANT: pull out samples from 'nytimes' that have matched dates to 'COVIDhub-ensemble' ...
df_hub,_ = load_csv('formatted_COVIDhub-ensemble')
my_dates = df_hub.Date.unique().tolist()
df = df[df.Date.isin(my_dates)]
df.reset_index(inplace=True,drop=True)

## reset index
df.reset_index(inplace=True,drop=True)

## SAVE CSV!
# fpath = os.path.join(DATA_DIR,'processed','scratch',f'formatted_nytimes-us-counties_{DT}.csv')
fpath = os.path.join(DATA_DIR,'processed',f'formatted_nytimes-us-counties_{DT}.csv')
df.to_csv(fpath,index=False)
print(DT)

## READ BACK AND CHECK
dff = pd.read_csv(fpath)
display(df, dff)
# dff.dtypes

Dropped rows with NaNs!
Num rows before: 2502832
Num rows after: 2421549
20241001-223952


,Fips,Date,True_cum_deaths
0,1001,20200613,6.0
1,1003,20200613,9.0
2,1005,20200613,1.0
3,1007,20200613,1.0
4,1009,20200613,1.0
...,...,...,...
313231,56037,20220507,126.0
313232,56039,20220507,16.0
313233,56041,20220507,39.0
313234,56043,20220507,44.0


,Fips,Date,True_cum_deaths
0,1001,20200613,6.0
1,1003,20200613,9.0
2,1005,20200613,1.0
3,1007,20200613,1.0
4,1009,20200613,1.0
...,...,...,...
313231,56037,20220507,126.0
313232,56039,20220507,16.0
313233,56041,20220507,39.0
313234,56043,20220507,44.0


# <font color=blue> ACS results

In [4]:
#####################################################
## ACS (CENSUS) RESULTS #############################
#####################################################

def get_state_acs_df(files):
    first_df = None
    for i,f in enumerate(files):
        this_df = pd.read_csv(f)
        if first_df is None:
            first_df = this_df
        else:
            df = pd.merge(first_df, this_df, on='GEOID', 
                          suffixes=(f'_x{i}', f'_x{i+1}'))
            first_df = df
    return df

def get_subdirs(path):
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

#####################################################

import glob
import os

dir_path = os.path.join(DATA_DIR,'raw','acs_results')

all_st_dfs = []
for i,(subdir, dirs, _) in enumerate(os.walk(dir_path)):
    
    if subdir==dir_path: 
        continue        
    else:
        postal = subdir[-2:]
        files = glob.glob(f'{subdir}/*.csv')
        df = get_state_acs_df(files)
        ## REFORMAT dataframe ... rename cols.
        df.rename({'GEOID':'Fips'},axis=1,inplace=True)
        all_st_dfs.append(df)
    
df = pd.concat(all_st_dfs)
df.reset_index(drop=True,inplace=True)

## check for cols with nans.
df = drop_sers_with_nans(df, from_axis='cols')

## reset index
df.reset_index(inplace=True,drop=True)

## SAVE CSV!
# fpath = os.path.join(DATA_DIR,'processed','scratch',f'formatted_acs_results_{DT}.csv')
fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_results_{DT}.csv')
df.to_csv(fpath,index=False)
print(DT)

## READ BACK AND CHECK
# dff = pd.read_csv(fpath)
# display(df, dff)

Dropped cols with NaNs!
Num cols before: 192
Num cols after: 191
20241001-223952


### <font color=blue> Add pop ratio to ACS and dedup identical covariates.

In [5]:
from epsampling.utils import drop_duplicate_cols

## Remove duplicate columns
df = drop_duplicate_cols(df)

## Rename cols
df.rename({'POP_x2':'Pop'},axis=1,inplace=True)
## Reorder columns
df = df[['Fips','Pop'] + [c for c in df.columns if c not in ['Fips','Pop']]]

# # # # # # # # # # # # # # # # # # # # # # # #
## Get county ratios and insert state pop, state fips, and county ratio cols.
# # # # # # # # # # # # # # # # # # # # # # # # 

df.insert(2, 'State_fips', 0)
df.insert(3, 'State_pop',0)
df.insert(4, 'Ratio', 0)

for tup in df.itertuples():
    
    state_fips = tup.Fips // 1000
    df.at[tup.Index, 'State_fips'] = state_fips    

for tup in df.itertuples():
    
    state_pop = sum(df[df.State_fips==tup.State_fips].Pop)
    ratio = tup.Pop / state_pop
    
    df.at[tup.Index, 'State_pop'] = state_pop
    df.at[tup.Index, 'Ratio'] = ratio
    
## reset cols
df.reset_index(inplace=True, drop=True)


## SAVE CSV!
# fpath = os.path.join(DATA_DIR,'processed','scratch',f'formatted_acs_pop_ratio_{DT}.csv')
fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_pop_ratio_{DT}.csv')
df.to_csv(fpath,index=False)
print(DT)

## READ BACK AND CHECK
dff = pd.read_csv(fpath)
display(df, dff)
# dff.dtypes

20241001-223952


,Fips,Pop,State_fips,State_pop,Ratio,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,...,HU_x14,POP_F,POP_AIANNH,POP_AIAN,EDU_HS,POP_A3564,POP_M,POP_16p_EMP_x7,POP_A25p,POP_NH
0,2013,3409,2,735951,0.004632,0,0,0,0,0,...,1113,1395,1501,1505,1648,1760,2014,2213,2598,3050
1,2016,5251,2,735951,0.007135,0,6,9,11,12,...,1456,2256,523,549,2626,2471,2995,3655,3838,4740
2,2020,292545,2,735951,0.397506,1842,2985,428,499,2642,...,118055,142897,19827,21150,94213,107273,149648,144598,193312,264762
3,2050,18514,2,735951,0.025157,32,90,250,239,9,...,5992,8790,15427,15640,6485,5770,9724,6454,10025,18131
4,2060,849,2,735951,0.001154,0,0,0,0,0,...,922,369,266,282,411,384,480,443,643,740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,42459,56,576641,0.073632,165,3135,0,30,73,...,19174,20609,454,506,16349,16838,21850,20884,27816,35626
3138,56039,23319,56,576641,0.040439,209,67,0,74,97,...,13255,11039,30,30,6148,9875,12280,14952,17659,19835
3139,56041,20514,56,576641,0.035575,0,748,0,8,62,...,8819,10077,20,52,8432,7709,10437,9688,13233,18533
3140,56043,7768,56,576641,0.013471,63,106,0,0,5,...,3842,3764,38,71,2992,2958,4004,3907,5423,6651


,Fips,Pop,State_fips,State_pop,Ratio,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,...,HU_x14,POP_F,POP_AIANNH,POP_AIAN,EDU_HS,POP_A3564,POP_M,POP_16p_EMP_x7,POP_A25p,POP_NH
0,2013,3409,2,735951,0.004632,0,0,0,0,0,...,1113,1395,1501,1505,1648,1760,2014,2213,2598,3050
1,2016,5251,2,735951,0.007135,0,6,9,11,12,...,1456,2256,523,549,2626,2471,2995,3655,3838,4740
2,2020,292545,2,735951,0.397506,1842,2985,428,499,2642,...,118055,142897,19827,21150,94213,107273,149648,144598,193312,264762
3,2050,18514,2,735951,0.025157,32,90,250,239,9,...,5992,8790,15427,15640,6485,5770,9724,6454,10025,18131
4,2060,849,2,735951,0.001154,0,0,0,0,0,...,922,369,266,282,411,384,480,443,643,740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,42459,56,576641,0.073632,165,3135,0,30,73,...,19174,20609,454,506,16349,16838,21850,20884,27816,35626
3138,56039,23319,56,576641,0.040439,209,67,0,74,97,...,13255,11039,30,30,6148,9875,12280,14952,17659,19835
3139,56041,20514,56,576641,0.035575,0,748,0,8,62,...,8819,10077,20,52,8432,7709,10437,9688,13233,18533
3140,56043,7768,56,576641,0.013471,63,106,0,0,5,...,3842,3764,38,71,2992,2958,4004,3907,5423,6651
